![image](https://github.com/Coding-Lane/Emojify-Text/blob/main/emoji.png?raw=true)

In [45]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [49]:
data = pd.read_csv('emoji_data.csv', header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [50]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [51]:
X = data[0].values
Y = data[1].values

# Embeddings

In [16]:
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [66]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [40]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)

10


In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [35]:
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [53]:
Ytrain = to_categorical(Y)

# Model

In [67]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [68]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [95]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),
    
    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [80]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 71s 77ms/step - loss: 1.5886 - accuracy: 0.2896
Epoch 2/100
6/6 [==============================] - 0s 30ms/step - loss: 1.5659 - accuracy: 0.3497
Epoch 3/100
6/6 [==============================] - 0s 27ms/step - loss: 1.5482 - accuracy: 0.3497
Epoch 4/100
6/6 [==============================] - 0s 32ms/step - loss: 1.5313 - accuracy: 0.3388
Epoch 5/100
6/6 [==============================] - 0s 32ms/step - loss: 1.5154 - accuracy: 0.3443
Epoch 6/100
6/6 [==============================] - 0s 28ms/step - loss: 1.4967 - accuracy: 0.3716
Epoch 7/100
6/6 [==============================] - 0s 25ms/step - loss: 1.4769 - accuracy: 0.3825
Epoch 8/100
6/6 [==============================] - 0s 33ms/step - loss: 1.4553 - accuracy: 0.3880
Epoch 9/100
6/6 [==============================] - ETA: 0s - loss: 1.4359 - accuracy: 0.42 - 0s 30ms/step - loss: 1.4336 - accuracy: 0.4153
Epoch 10/100
6/6 [==============================] - 0s 23ms/step - loss: 1.

6/6 [==============================] - 0s 23ms/step - loss: 0.2231 - accuracy: 0.9945
Epoch 84/100
6/6 [==============================] - 0s 31ms/step - loss: 0.2175 - accuracy: 0.9945
Epoch 85/100
6/6 [==============================] - 0s 26ms/step - loss: 0.2120 - accuracy: 1.0000
Epoch 86/100
6/6 [==============================] - 0s 25ms/step - loss: 0.2070 - accuracy: 1.0000
Epoch 87/100
6/6 [==============================] - 0s 26ms/step - loss: 0.2030 - accuracy: 1.0000
Epoch 88/100
6/6 [==============================] - 0s 25ms/step - loss: 0.1991 - accuracy: 1.0000
Epoch 89/100
6/6 [==============================] - 0s 29ms/step - loss: 0.1953 - accuracy: 1.0000
Epoch 90/100
6/6 [==============================] - 0s 34ms/step - loss: 0.1917 - accuracy: 1.0000
Epoch 91/100
6/6 [==============================] - 0s 33ms/step - loss: 0.1887 - accuracy: 1.0000
Epoch 92/100
6/6 [==============================] - 0s 28ms/step - loss: 0.1855 - accuracy: 1.0000
Epoch 93/100
6/6 [=====

In [84]:
test = ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️
